# Learning the flow field of Stokes flow

This example demonstrates how to train the MeshGraphNet model to learn the flow field
of Stokes flow and further
improve the accuary of the model predictions by physics-informed inference. This example
also demonstrates how to use physics utilites from
[PhysicsNeMo-Sym](https://github.com/NVIDIA/physicsnemo-sym) to introduce physics-based
constraints.


## Problem overview

The partial differential equation is defined as

$$\begin{aligned}
    -\nu \Delta \mathbf{u} +\nabla p=0, \\
    \nabla \cdot \mathbf{u} = 0,
\end{aligned}$$

where $\mathbf{u} = (u, v)$ defines the velocity and $p$ the pressure, and $\nu$ is the
kinematic viscosity.
The underlying geometry is a pipe without a polygon. On the inlet
$\Gamma_3=0 \times[0,0.4]$, a parabolic inflow profile is prescribed,

$$\begin{aligned}
    \mathbf{u}(0, y)= \mathbf{u}_{\mathrm{in}} =
    \left(\frac{4 U y(0.4-y)}{0.4^2}, 0\right)
\end{aligned}$$

with a maximum velocity $U=0.3$. On the outlet $\Gamma_4=2.2 \times[0,0.4]$, we
define the outflow condition

$$\begin{aligned}
    \nu \partial_\mathbf{n} \mathbf{u}-p \mathbf{n}=0,
\end{aligned}$$

where $\mathbf{n}$ denotes the outer normal vector.

Our goal is to train a MeshGraphNet (MGN) to learn the map from the polygon geometry to the
velocity and pressure field.
However, sometimes data-driven models may not be able to yield reasonable predictive
accuracy due to network capacity or limited dataset. We can fine-tune our results
using PINNs or directly fine-tune MGN when the PDE is available. The fine-tuning during inference is much faster
than training the PINN or MGN model from the scratch as the model has a better initialization
from the data-driven training.

For the fine-tuning step, we formulate two losses. First loss is to match the
predictions of the original MeshGraphNet model. Second loss includes the physics
losses, i.e. the PDE residuals and the boundary conditions. Having the data loss
helps the PINN model converge faster than training from scratch.

## Prerequisites

Install the requirements using:

```bash
pip install -r requirements.txt
pip install  dgl -f https://data.dgl.ai/wheels/torch-2.4/cu124/repo.html --no-deps
pip install nvidia-physicsnemo.sym --no-build-isolation
```

## Dataset

Our dataset provides  numerical simulations of Stokes flow in a pipe domain obstructed
by a random polygon. It contains 1000 random samples and all the simulations were
performed using Fenics. For each sample, the numerical solution cotains the mesh and
the flow information about velocity, pressure, and markers identifying different
boundaries within the domain.

To download the full dataset, please run the bash script in `raw_dataset`

```bash
bash download_dataset.sh
```

In [1]:
!cd raw_dataset && bash download_dataset.sh

download_dataset.sh: line 18: $'\nDownload Stokes flow dataset\n': command not found
--2025-08-21 04:03:03--  https://api.ngc.nvidia.com/v2/resources/org/nvidia/team/physicsnemo/physicsnemo_datasets_stokes_flow/0.0.1/files?redirect=true&path=results_polygon.zip
Resolving api.ngc.nvidia.com (api.ngc.nvidia.com)... 44.237.192.76, 44.224.84.187
connected. to api.ngc.nvidia.com (api.ngc.nvidia.com)|44.237.192.76|:443... 
HTTP request sent, awaiting response... 302 Found
Location: https://xfiles.ngc.nvidia.com/org/nvidia/team/physicsnemo/recipes/physicsnemo_datasets_stokes_flow/versions/0.0.1/files/results_polygon.zip?versionId=XL1XOF1guLrrzUFYfuAtpbJm4v4Uu4Ib&Expires=1755835383&Signature=avgScbcZCLDEglpAkBKWKzhxk6pJUsAj8qVB~FzLa8SrTo3fa4bp87HBw6nTSxy7o5JHNBKFQFAvOx6zebAgljKrQfIHaz99CTdeGjNuCLdXMntL3gz4a630qZ1VJiwS2qCFjQKF5JVlbHBbhNT9gjbKxaJp0cV~7i5nprHLS7aGghTKoQquwvsf99xBgQa62HYPCNB51KL2cOovl5jeBZayiFkhSmwTtu3qlF0qQDYkgYK~jeqPGmiZeEufLopsAJp75HsNdUUFo8r9NEt0SUGK2-~-vFB9k3cBwaN4x9qnYhrYkUc

## Model overview and architecture

 The inputs of our MeshGraphNet model is:

- mesh

Output of the MeshGraphNet model are:

- velocity field pressure
- pressure field

The input to the model is in form of a `.vtp` file and is then converted to
bi-directional DGL graphs in the dataloader. The final results are also written in the
form of `.vtp` files in the inference code. A hidden dimensionality of 256 is used in
the encoder, processor, and decoder. The encoder and decoder consist of two hidden
layers, and the processor includes 15 message passing layers. Batch size per GPU is
set to 1. Summation aggregation is used in the
processor for message aggregation. A learning rate of 0.0001 is used, decaying
exponentially with a rate of 0.99985.

![Comparison of the MeshGraphNet prediction and the filetered prediction against the
ground truth for velocity and pressure for one
of the samples from the test dataset.](../../../docs/img/stokes.png)

## Getting Started

Once you've download the dataset, follow these steps to preprocess it:

**Run the Preprocessing Script**: Execute the provided script to process the dataset.
This will distribute the data
randomly across three directories: `training`, `validation`, and `test`.

```bash
python preprocess.py
````

In [2]:
!python preprocess.py

In [4]:
!ls ./dataset/

test  train  validation


## Training the model 

To train the model, run

```bash
python train.py
```

Use GraphDataloader and Meshgraphnet model
```python
# instantiate validation dataloader
training_dataloader = GraphDataLoader(
            validation_dataset,
            batch_size=cfg.batch_size,
            shuffle=False,
            drop_last=True,
            pin_memory=True,
            use_ddp=False,
        )

# instantiate the model
model = MeshGraphNet(
            cfg.input_dim_nodes,
            cfg.input_dim_edges,
            cfg.output_dim,
            aggregation=cfg.aggregation,
            hidden_dim_node_encoder=cfg.hidden_dim_node_encoder,
            hidden_dim_edge_encoder=cfg.hidden_dim_edge_encoder,
            hidden_dim_node_decoder=cfg.hidden_dim_node_decoder,
        )
```

In [ ]:
!python train.py

/usr/local/lib/python3.12/dist-packages/physicsnemo/distributed/manager.py:403: UserWarning: Could not initialize using ENV, SLURM or OPENMPI methods. Assuming this is a single process job
  warn(
Preparing the train dataset...
/usr/local/lib/python3.12/dist-packages/physicsnemo/datapipes/gnn/stokes_dataset.py:160: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  disp = torch.tensor(pos[row.long()] - pos[col.long()])
Preparing the validation dataset...
[2025-08-21 04:15:16,087][main][INFO] - Using FusedAdam optimizer
[2025-08-21 04:15:16,088][checkpoint][WARNING] - Provided checkpoint directory /workspace/physicsnemo/examples/cfd/stokes_mgn/checkpoints does not exist, skipping load
[2025-08-21 04:15:16,088][main][INFO] - Training started...
[2025-08-21 04:15:39,317][main][INFO] - epoch: 0, loss:  5.469e-01, lr: 9.277382672642156e-05, time 

# Inference
The URL to the dashboard will be displayed in the terminal after the run is launched.
Alternatively, the logging utility in `train.py` can be switched to MLFlow.

Once the model is trained, run

```bash
python inference.py
```

In [6]:
!python inference.py

/usr/local/lib/python3.12/dist-packages/google/protobuf/internal/well_known_types.py:91: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  _EPOCH_DATETIME_NAIVE = datetime.datetime.utcfromtimestamp(0)
[2025-08-21 11:43:17,087][main][INFO] - Rollout started...
Using cuda device
Preparing the test dataset...
/usr/local/lib/python3.12/dist-packages/physicsnemo/datapipes/gnn/stokes_dataset.py:160: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  disp = torch.tensor(pos[row.long()] - pos[col.long()])
[2025-08-21 11:43:17,659][checkpoint][WARNING] - `DistributedManager` not initialized already. Initializing now, but this might lead to unexpected errors
/usr/local

![Comparison of the MeshGraphNet prediction and the filetered prediction against the
ground truth for velocity and pressure for one
of the samples from the test dataset.](./images/output_p_comparison.png)

![Comparison of the MeshGraphNet prediction and the filetered prediction against the
ground truth for velocity and pressure for one
of the samples from the test dataset.](./images/output_u_comparison.png)

![Comparison of the MeshGraphNet prediction and the filetered prediction against the
ground truth for velocity and pressure for one
of the samples from the test dataset.](./images/output_v_comparison.png)

## Finetuning with physics

Use PhysicsNeMo Sym's PhysicsInformer utility for computing PDE losses

```python
self.node_pde = Stokes(nu=self.nu, dim=2)

self.phy_informer = PhysicsInformer(
        required_outputs=["continuity", "momentum_x", "momentum_y"],
        equations=self.node_pde,
        grad_method="autodiff",
        device=self.device,
        )
        
results_int = self.phy_informer.forward(
            {
                "coordinates": self.coords,
                "u": model_out["u"],
                "v": model_out["v"],
                "p": model_out["p"],
            }
        )
```

## Approach 1: Using PINNs to finetune
#### Step 1: Create a PINN model using Deep Neural network and pass the mesh coordinates

```python
class MdlsSymDNN(Arch):

    def __init__(
        self,
        input_keys=[Key("x"), Key("y")],
        output_keys=[Key("u"), Key("v"), Key("p")],
        layers=[2, 128, 128, 128, 128, 3],
        fourier_features=64,
    ):
        super().__init__(
            input_keys=input_keys,
            output_keys=output_keys,
        )

        self.mdls_model = DNN(layers, fourier_features)

    def forward(self, dict_tensor: Dict[str, torch.Tensor]):
        x = self.concat_input(
            dict_tensor,
            self.input_key_dict,
            detach_dict=self.detach_key_dict,
            dim=-1,
        )
        out = self.mdls_model(x)
        return self.split_output(out, self.output_key_dict, dim=1)

```

#### Step 2: Setup Loss function 

Data loss : GNN results - PINN prediction
```python
# data loss
loss_u = torch.mean((self.gnn_u - pred_u) ** 2)
loss_v = torch.mean((self.gnn_v - pred_v) ** 2)
loss_p = torch.mean((self.gnn_p - pred_p) ** 2)
```

 Total Loss= Data loss + PDE Loss + Boundary Loss

Make sure the grad method in PhysicsInformer is autodiff
```python
self.phy_informer = PhysicsInformer(
        required_outputs=["continuity", "momentum_x", "momentum_y"],
        equations=self.node_pde,
        grad_method="autodiff",
        device=self.device,
        )
```

For PINN based fine-tuning the model using physics-informed learning, run

```bash
python pi_fine_tuning.py
```

In [7]:
!python pi_fine_tuning.py

[2025-08-21 11:43:27,709][main][INFO] - Inference (with physics-informed training for fine-tuning) started...
[2025-08-21 11:43:28,085][main][INFO] - Iteration: 0
[2025-08-21 11:43:28,085][main][INFO] - Loss u: 6.041e-02
[2025-08-21 11:43:28,085][main][INFO] - Loss v: 6.013e-03
[2025-08-21 11:43:28,085][main][INFO] - Loss p: 1.524e-01
[2025-08-21 11:43:28,085][main][INFO] - Loss u_in: 6.043e-02
[2025-08-21 11:43:28,086][main][INFO] - Loss v_in: 4.784e-03
[2025-08-21 11:43:28,086][main][INFO] - Loss u noslip: 8.809e-04
[2025-08-21 11:43:28,086][main][INFO] - Loss v noslip: 4.113e-03
[2025-08-21 11:43:28,086][main][INFO] - Loss momentum u: 2.574e-02
[2025-08-21 11:43:28,086][main][INFO] - Loss momentum v: 1.234e-02
[2025-08-21 11:43:28,086][main][INFO] - Loss continuity: 1.879e-02
[2025-08-21 11:43:28,088][main][INFO] - Error u: 1.070e+00
[2025-08-21 11:43:28,089][main][INFO] - Error v: 1.871e+00
[2025-08-21 11:43:28,089][main][INFO] - Error p: 1.241e+00
[2025-08-21 11:43:28,089][main][I

![Comparison of the MeshGraphNet prediction and the filetered prediction against the
ground truth for velocity and pressure for one
of the samples from the test dataset.](./images/output_p_comparison_pinn.png)

![Comparison of the MeshGraphNet prediction and the filetered prediction against the
ground truth for velocity and pressure for one
of the samples from the test dataset.](./images/output_u_comparison_pinn.png)

![Comparison of the MeshGraphNet prediction and the filetered prediction against the
ground truth for velocity and pressure for one
of the samples from the test dataset.](./images/output_v_comparison_gnn.png)

## Approach 2: Finetuning MeshGraphnet model directly

Model is Meshgraphnet instead of DNN
```python
self.model = MeshGraphNet(
    cfg.input_dim_nodes
    + 128,  # additional 128 node features from fourier features
    cfg.input_dim_edges,
    cfg.output_dim,
    aggregation=cfg.aggregation,
    hidden_dim_node_encoder=cfg.hidden_dim_node_encoder,
    hidden_dim_edge_encoder=cfg.hidden_dim_edge_encoder,
    hidden_dim_node_decoder=cfg.hidden_dim_node_decoder,
).to(self.device)
```

Now the derivatives are computed using least squares method using connectivity tensor on unstructured grids

```python
self.phy_informer = PhysicsInformer(
    required_outputs=["continuity", "momentum_x", "momentum_y"],
    equations=self.node_pde,
    grad_method="least_squares",
    device=self.device,
    compute_connectivity=False,
)
```

For MGN based fine-tuning the model using physics-informed learning, run

```bash
python pi_fine_tuning_gnn.py
```

In [8]:
!python pi_fine_tuning_gnn.py

/workspace/physicsnemo/examples/cfd/stokes_mgn/utils.py:149: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  disp = torch.tensor(pos[row.long()] - pos[col.long()])
[2025-08-21 11:47:58,629][main][INFO] - Inference (with physics-informed training for fine-tuning) started...
[2025-08-21 11:47:59,460][main][INFO] - Iteration: 0
[2025-08-21 11:47:59,460][main][INFO] - Loss u: 1.454e-02
[2025-08-21 11:47:59,460][main][INFO] - Loss v: 1.023e-01
[2025-08-21 11:47:59,460][main][INFO] - Loss p: 7.403e-02
[2025-08-21 11:47:59,461][main][INFO] - Loss u_in: 9.543e-03
[2025-08-21 11:47:59,461][main][INFO] - Loss v_in: 1.029e-01
[2025-08-21 11:47:59,461][main][INFO] - Loss u noslip: 3.166e-02
[2025-08-21 11:47:59,461][main][INFO] - Loss v noslip: 1.288e-01
[2025-08-21 11:47:59,461][main][INFO] - Loss momentum u: 6.479e+00
[2025-08-21 11:47:59,461][main

![Comparison of the MeshGraphNet prediction and the filetered prediction against the
ground truth for velocity and pressure for one
of the samples from the test dataset.](./images/output_p_comparison_gnn.png)

![Comparison of the MeshGraphNet prediction and the filetered prediction against the
ground truth for velocity and pressure for one
of the samples from the test dataset.](./images/output_u_comparison_gnn.png)

![Comparison of the MeshGraphNet prediction and the filetered prediction against the
ground truth for velocity and pressure for one
of the samples from the test dataset.](./images/output_v_comparison_gnn.png)